In [ ]:
##La documentación de Methylartist es un horror. Se puede ejecutar directamente contra los BAMs.
##Para crear un entorno:

mamba create -n methylartist methylartist

In [ ]:
##Por cierto, creé un par de directorios para separa las cosas, ahora la pipeline está dentro de uno que se llama "ont-pipeline" y el otro "6mA" 
##para poner todo lo que hagas en relación a eso.

$ pwd
/storage/scratch01/groups/bu/teloseq/

$ ls
6mA  ont-pipeline

In [ ]:
##Environment created at: 

$ cd /storage/scratch01/users/mespejo/methylartist

$ mamba create -n methylartist methylartist
$ conda activate methylartist 

##Info conda vs mamba: Los de conda no aceptaron cambios para acelerar su software, que a esa altura era inutilizable en entornos grandes
por lo que tardaba en resolver dependencias, y entonces los que proponían los cambios crearon mamba. Es 100% compatible y 
hay que usarlo para todo lo que soporta, pero algunos comandos (por ejemplo activate) son más fáciles de usar con conda aún.


In [ ]:
## The methylartist tool requires an indexed BAM file (i.e., with a corresponding .bai index). To create the index, use samtools:

$ samtools index /storage/scratch01/groups/bu/teloseq/methylartist/data/IE11_multiplex/IE11supv435mC6mAfilter_aligned.bam

In [ ]:
##Output directory created 

export METHYLARTIST_SAMPLE=T/storage/scratch01/users/mespejo/methylartist/DiMeLoseqH3K27me3Teloseq180624/DiMeLoseq_H3K27me3_Teloseq_supv56mAall.bam 
export METHYLARTIST_OUTDIR=/storage/scratch01/groups/bu/teloseq/6mA/output/DiMeLoseqH3K27me3Teloseq180624/priorTeloseq/
export METHYLARTIST_WORKDIR=/storage/scratch01/groups/bu/teloseq/6mA/work/DiMeLoseqH3K27me3Teloseq180624/priorTeloseq/
export NXF_SINGULARITY_CACHEDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/singularity/cache/ 
export METHYLARTIST_INPUT=/storage/scratch01/users/mespejo/methylartist/DiMeLoseqH3K27me3Teloseq180624/DiMeLoseq_H3K27me3_Teloseq_supv56mAall.bam
export METHYLARTIST_REFERENCE=/storage/scratch01/users/mespejo/methylartist/DiMeLoseqH3K27me3Teloseq180624/YAOqpMP_reference.fasta.gz

##Como reference tengo que coger la referencia con la que el .bam file está mapeado. Ultimamente para el mapping con minimap2 utilizamos:

export METHYLARTIST_REFERENCE=/storage/scratch01/groups/bu/teloseq/dimeloseq/reference/hg002v1.1.fasta

##Esta refencia tiene esta pinta:

chr1_MATERNAL   4       *       0       0       *       *       0       0       CCCTAACCCCTAACCCCTAACCCCTAACCCCTAACCCCTAACCCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCC

##Las referencias que utiliza Telo-seq de YAOqpMP_reference.fasta.gz:

GWHDQZJ00000001_p       4       *       0       0       *       *       0       0       ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA

Y HG002qpMP_reference.fasta.gz
chr1_PATERNAL_P 4       *       0       0       *       *       0       0       TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA

##Las referencias que usan de ejemplo en methyartist con este comando (-r Homo_sapiens_assembly38.fasta) tienen esta pinta:
    
methylartist region -i chr11:100956822-101228191 -d MCF7_data_megalodon.txt -g Homo_sapiens.GRCh38.97.chr.sorted.gtf.gz -p 32 -n CG -r Homo_sapiens_assembly38.fasta --samplepalette magma -l 101126888-101129371 --highlightpalette viridis
chr1    4       *       0       0       *       *       0       0       NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

##Así que con nuestra referencia podríamos hacer lo mismo, indicar la región con:

    -i chr1_MATERNAL:0000-5700 -r hg002v1.1.fasta (5700 es donde está el corte con ECORV).

##Lo que no se si podríamos extender para ver la longitud de telómeros. 
##O para poner como referencia siempre el corte de la región subtelomérica. ¿Tendría que mirarlo cromosoma a cromosoma?


In [ ]:
##Luego el comando es:

methylartist region \
         -b my.bam \
         -r myref.fa \
         -i chr1:1000-2000 \
         -p 16 # numero de cores \
         -n A or CG # mod for Adenine or CpG methylation \
         -o myplot.png

methylartist region -b $METHYLARTIST_SAMPLE -r METHYLARTIST_REFERENCE -i chr1_MATERNAL:0000-5700 -p 16 -n A -work-dir $METHYLARTIST_WORKDIR --out_dir $METHYLARTIST_OUTDIR -o chr1qpaternal.plot

In [ ]:
$ export METHYLARTIST_SAMPLE=/storage/scratch01/groups/bu/teloseq/methylartist/data/IE11_multiplex/IE11supv435mC6mAfilter_aligned.bam
$ export METHYLARTIST_INPUT=/storage/scratch01/groups/bu/teloseq/methylartist/data/IE11_multiplex/IE11supv435mC6mAfilter_aligned.bam
$ export METHYLARTIST_OUTDIR=/storage/scratch01/groups/bu/teloseq/methylartist/output/IE11_multiplex/
$ export METHYLARTIST_WORKDIR=/storage/scratch01/groups/bu/teloseq/methylartist/work/IE11_multiplex/
$ export NXF_SINGULARITY_CACHEDIR=/storage/scratch01/groups/bu/teloseq/ont-pipeline/singularity/cache/ 
$ export METHYLARTIST_REFERENCE=/storage/scratch01/groups/bu/teloseq/methylartist/data/hg002v1.1.fasta 

srun -c 12 --mem=32000 -t 300 --pty bash -c "methylartist region -b $METHYLARTIST_SAMPLE -r $METHYLARTIST_REFERENCE -i chr1_MATERNAL:0000-5700 -p 16 -n CG -o /storage/scratch01/groups/bu/teloseq/methylartist/output/IE11_multiplex/IE11_chr1maternalp.plot"

In [ ]:
##Usando reference genome YAOqpMP_reference.fasta
srun -c 12 --mem=32000 -t 300 --pty bash -c "methylartist region -b ./DiMeLoseqH3K27me3Teloseq180624/DiMeLoseq_H3K27me3_Teloseq_supv56mAall.bam -r ./YAOqpMP_reference.fasta -i chr1:0-15000 -p 16 -n A -o chr1q.plot"

In [ ]:
##Usando mejor reference genome HG002qpMP_reference.fasta
srun -c 12 --mem=32000 -t 300 --pty bash -c "methylartist region -b ./DiMeLoseqH3K27me3Teloseq180624/DiMeLoseq_H3K27me3_Teloseq_supv56mAall.bam -r ./HG002qpMP_reference.fasta -i chr1_PATERNAL_P -p 16 -n A -o chr1q.plot"

##Sí, lo de Methylartist es por la referencia. Vas a tener que ajustar la  región a la referencia que estés usando. Imagino que no hay chr1 en tu fasta.

srun -c 12 --mem=32000 -t 300 --pty bash -c "methylartist region -b ./DiMeLoseqH3K27me3Teloseq180624/DiMeLoseq_H3K27me3_Teloseq_supv56mAall.bam -r ./Homo_sapiens_assembly38.fasta -i chr1:0-15000 -p 16 -n A -o chr1q.plot"

In [ ]:
#El "no targets in header" cuando corres samtools quickcheck para los BAM que se generan es porque es un BAM sin alinear, o sea un abuso del formato. Pero es lo correcto para la salida de Dorado.

#Los errores del out son normales: primero falla la descarga del modelo  con wget, y lo reintenta y lo logra con curl. Luego fallan 3 reads de 
#las quichicientasmil, así que sin importancia.

#Lo único que te recomendaría sería tomar no solo podt_pass, sino también pod5_fail (poniendo los contenidos de ambas en un único directorio). 
#Porque esa categorización se hizo durante el sequencing con el modelo fast, y quizá el sup puede rescatar algo de las fail. En todo caso no 
#tires el bam de sólo pass, que es interesante comparar.exi

In [ ]:
##Alignment with dorado of the demultiplexed files
#where index is a reference to align to in (FASTQ/FASTA/.mmi) format and reads is a folder or file in any HTS format.

sbatch -p gpu --gres=gpu:A100:1 -t 300 --mem=32G --wrap "/storage/scratch01/users/mespejo/Dorado/bin/dorado aligner /storage/scratch01/users/mespejo/methylartist/references/HG002qpMP_reference.fasta Telo-seq_barcode01.bam > ./IE11_demultiplex_bam_align/Telo-seq_barcode01_aligned.bam"

export METHYLARTIST_SAMPLE=/storage/scratch01/users/mespejo/methylartist/data/IE11_Teloseq_Multiplexing_01/IE11_Teloseq_demultiplex/IE11_demultiplex_bam/IE11_demultiplex_bam_align/Telo-seq_barcode01_aligned.bam 
export METHYLARTIST_OUTDIR=/storage/scratch01/users/mespejo/methylartist/output/IE11_Teloseq_Multiplexing_01/IE11_Teloseq_demultiplex/
export METHYLARTIST_WORKDIR=/storage/scratch01/users/mespejo/methylartist/work/
export METHYLARTIST_INPUT=/storage/scratch01/users/mespejo/methylartist/data/IE11_Teloseq_Multiplexing_01/IE11_Teloseq_demultiplex/IE11_demultiplex_bam/IE11_demultiplex_bam_align/Telo-seq_barcode01_aligned.bam 
export METHYLARTIST_REFERENCE=/storage/scratch01/users/mespejo/methylartist/data/references/HG002qpMP_reference.fasta

srun -c 12 --mem=32000 -t 300 --pty bash -c "methylartist region -b ./DiMeLoseqH3K27me3Teloseq180624/DiMeLoseq_H3K27me3_Teloseq_supv56mAall.bam -r ./HG002qpMP_reference.fasta -i chr1:0000-15000 -p 16 -n A -work-dir $METHYLARTIST_WORKDIR --out_dir $METHYLARTIST_OUTDIR -o chr1qpaternal.plot"

srun -c 12 --mem=32000 -t 300 --pty bash -c "methylartist region -b /storage/scratch01/users/mespejo/methylartist/data/IE11_Teloseq_Multiplexing_01/IE11_Teloseq_demultiplex/IE11_demultiplex_bam/IE11_demultiplex_bam_align/Telo-seq_barcode01_aligned.bam -r HG002qpMP_reference.fasta -i chr1_PATERNAL_P -n CG -p 16 # numero de cores -n A -o /storage/scratch01/users/mespejo/methylartist/output/IE11_Teloseq_Multiplexing_01/IE11_Teloseq_demultiplex/chr1_PATERNAL_P.png"


In [ ]:
2025-01-17 14:41:37,998 starting methylartist with command: /home/mespejo/miniforge3/envs/methylartist/bin/methylartist region -h
usage: methylartist region [-h] -i INTERVAL [-d DATA] [-b BAMS] -n MOTIF -r REF [-g GTF] [-l HIGHLIGHT] [-w WINDOWS] [-p PROCS] [-m MODS] [-s SMOOTHWINDOWSIZE] [-q MIN_MAPQ] [-c PLOT_COVERAGE] [-o OUTFILE] [--logcover] [--allreads] [--highlight_bed HIGHLIGHT_BED] [--motifsize MOTIFSIZE]
                           [--maxuncovered MAXUNCOVERED] [--modspace MODSPACE] [--readmask READMASK] [--min_window_calls MIN_WINDOW_CALLS] [--smoothfunc SMOOTHFUNC] [--smoothalpha SMOOTHALPHA] [--eff EFF] [--ymin YMIN] [--ymax YMAX] [--cover_ymin COVER_YMIN] [--max_read_density MAX_READ_DENSITY]
                           [--samplepalette SAMPLEPALETTE] [--coverpalette COVERPALETTE] [--highlightpalette HIGHLIGHTPALETTE] [--genepalette GENEPALETTE] [--gene_track_height GENE_TRACK_HEIGHT] [--highlight_alpha HIGHLIGHT_ALPHA] [--highlight_centerline HIGHLIGHT_CENTERLINE] [--panelratios PANELRATIOS]
                           [--nticks NTICKS] [--skip_align_plot] [--force_align_plot] [--genes GENES] [--labelgenes] [--show_transcripts] [--exonheight EXONHEIGHT] [--width WIDTH] [--height HEIGHT] [--phased] [--primary_only] [--color_by_hp] [--scale_fullwidth SCALE_FULLWIDTH] [--svg]

options:
  -h, --help            show this help message and exit
  -i INTERVAL, --interval INTERVAL
                        chrom:start-end
  -d DATA, --data DATA  text file with .bam filename and corresponding methylation database per line(whitespace-delimited)
  -b BAMS, --bams BAMS  one or more .bams with MM and ML tags for modification calls (see samtags spec)
  -n MOTIF, --motif MOTIF
                        normalise window sizes to motif occurance
  -r REF, --ref REF     ref genome fasta, required if normalising windows with -n/--norm_motif
  -g GTF, --gtf GTF     genes or intervals to display in gtf format
  -l HIGHLIGHT, --highlight HIGHLIGHT
                        format: start-end, (can be chrom:start-end but chrom is ignored) can comma-delimit multiple highlights
  -w WINDOWS, --windows WINDOWS
                        set window count, default=auto
  -p PROCS, --procs PROCS
                        multiprocessing
  -m MODS, --mods MODS  mods to consider (comma-delimited, default = all available)
  -s SMOOTHWINDOWSIZE, --smoothwindowsize SMOOTHWINDOWSIZE
                        size of window for smoothing (default=auto)
  -q MIN_MAPQ, --min_mapq MIN_MAPQ
                        minimum mapping quality (mapq), default = 10
  -c PLOT_COVERAGE, --plot_coverage PLOT_COVERAGE
                        plot coverage from bam(s) (can be comma-delimited list)
  -o OUTFILE, --outfile OUTFILE
                        output file name (default: generated from input)
  --logcover            apply log2(count+1) to coverage data (--plot_coverage)
  --allreads            show all alignments (secondary/supplementary alignments hidden by default)
  --highlight_bed HIGHLIGHT_BED
                        BED3+1 format (chrom, start, end, optional_colour) where colour (optional) must be intelligible to matplotlib
  --motifsize MOTIFSIZE
                        mod motif size, only used with -b/--bams (default is 2 as "CG" is most common use case, e.g. set to 1 for 6mA)
  --maxuncovered MAXUNCOVERED
                        maximum percentage of uncovered windows tolerated (default = 50.0)
  --modspace MODSPACE   increase to increase spacing between links in top panel (default=auto)
  --readmask READMASK   mask reads from being shown in interval(s) (start-end or chrom:start-end; chrom ignored). Can be comma-delimited.
  --min_window_calls MIN_WINDOW_CALLS
                        minimum reads per window to include in plot (default = 1)
  --smoothfunc SMOOTHFUNC
                        smoothing function, one of: flat,hanning,hamming,bartlett,blackman (default = hanning)
  --smoothalpha SMOOTHALPHA
                        alpha (transparency) value for smoothed plot (default = 1.0)
  --eff EFF             conversion efficiency (for e.g. bs-seq or em-seq), input as comma-delimited sample:eff e.g. MySample1.m:0.9,MySample2.m:0.8
  --ymin YMIN           y-axis minimum for smoothed plot (default = -0.05)
  --ymax YMAX           y-axis maximum for smoothed plot (default = 1.05
  --cover_ymin COVER_YMIN
                        y-axis minimum for coverage plot (default = 0)
  --max_read_density MAX_READ_DENSITY
                        filter reads with call density greater >= value, can be helpful in footprinting assays (default=None)
  --samplepalette SAMPLEPALETTE
                        palette for samples (default = "tab10"), see https://seaborn.pydata.org/tutorial/color_palettes.html
  --coverpalette COVERPALETTE
                        colour palette name for coverage plot (default = "mako")
  --highlightpalette HIGHLIGHTPALETTE
                        colour palette name for highlights (default = "Blues")
  --genepalette GENEPALETTE
                        colour palette name for highlights (default = "viridis")
  --gene_track_height GENE_TRACK_HEIGHT
                        maximum number of gene track layers
  --highlight_alpha HIGHLIGHT_ALPHA
                        alpha for highlighting in panels (between 0 and 1, default = 0.25)
  --highlight_centerline HIGHLIGHT_CENTERLINE
                        change highlight to line (specify width)
  --panelratios PANELRATIOS
                        Alter panel ratios: needs to be 4 (or 5 if --plot_coverage) comma-seperated integers. Default: 1,5,3,3
  --nticks NTICKS       tick count (default=10)
  --skip_align_plot     blank alignment plot, useful if unneeded or for runtime.
  --force_align_plot    retain alignment plot even over regions > 5Mbp where it would be disabled automatically
  --genes GENES         genes of interest (comma delimited)
  --labelgenes          plot gene names
  --show_transcripts    plot all transcripts, use transcript_id/transcript_name attrs
  --exonheight EXONHEIGHT
                        set exon height (default=0.8)
  --width WIDTH         image width (inches, default=16)
  --height HEIGHT       image width (inches, default=8)
  --phased              currently only considers two phases (diploid)
  --primary_only        ignore non-primary alignments
  --color_by_hp         color samples by HP value (req --phased)
  --scale_fullwidth SCALE_FULLWIDTH
                        scale plot output relative to value (e.g. use length of chrom 1)

In [ ]:
##Methylartist to count %methylation in a region

In [ ]:
##We first create a .bed file with excel. We save it as a tab-text .txt and then we change it to .bed
chr1_MATERNAL   2415    4915
chr1_MATERNAL   244014790       244017290
chr1_PATERNAL   3418    5918
chr1_PATERNAL   252053115       252055615
chr2_MATERNAL   2687    5187
chr2_MATERNAL   242107695       242110195
chr2_PATERNAL   1945    4445
chr2_PATERNAL   241866969       241869469
chr3_MATERNAL   201086583       201089083
chr3_PATERNAL   201502801       201505301
chr4_MATERNAL   1803    4303
chr4_MATERNAL   191665141       191667641
chr4_PATERNAL   2135    4635
chr4_PATERNAL   192373651       192376151
chr5_MATERNAL   183255136       183257636
chr5_PATERNAL   188868831       188871331
chr6_MATERNAL   1443    3943
chr6_MATERNAL   174736176       174738676
chr6_PATERNAL   2234    4734
chr6_PATERNAL   174404856       174407356
chr7_MATERNAL   2381    4881
chr7_MATERNAL   160950141       160952641
chr7_PATERNAL   2084    4584
chr7_PATERNAL   160099884       160102384

First colum is chr name according to reference (hg002v1.1.) then start of the region and end.

    

In [ ]:
##Then we use methylartist segmeth
 methylartist segmeth 
-b /storage/scratch01/groups/bu/teloseq/methylartist/data/IE11_multiplex/IE11supv435mC6mAfilter_aligned.bam 
-i ./chr1_7_2500.bed
--motif CG 
--ref /storage/scratch01/groups/bu/teloseq/methylartist/data/hg002v1.1.fasta  
-p 32 
-o /storage/scratch01/groups/bu/teloseq/methylartist/output/IE11_multiplex/IE11_multiplex_segmeth.tsv

srun -c 12 --mem=32000 -t 300 --pty bash -c "methylartist segmeth -b /path/to/bamfile  -i ./data/chr1_7_3000.bed --motif CG --ref /storage/scratch01/groups/bu/teloseq/methylartist/data/hg002v1.1.fasta -p 32 -o ./output/path/to/file"

In [ ]:
usage: methylartist segmeth [-h] [-d DATA] [-b BAMS] -i INTERVALS [-p PROCS] [-q MIN_MAPQ] [-o OUTFILE] [--ref REF] [--motif MOTIF]
                            [--max_read_density MAX_READ_DENSITY] [--excl_ambig] [--spanning_only] [--primary_only] [--phased]

options:
  -h, --help            show this help message and exit
  -d DATA, --data DATA  text file with .bam filename and corresponding methylation database per line(whitespace-delimited)
  -b BAMS, --bams BAMS  one or more .bams with Mm and Ml tags for modification calls (see samtags spec)
  -i INTERVALS, --intervals INTERVALS
                        .bed file
  -p PROCS, --procs PROCS
                        multiprocessing
  -q MIN_MAPQ, --min_mapq MIN_MAPQ
                        minimum mapping quality (mapq), default = 10
  -o OUTFILE, --outfile OUTFILE
                        output file name (default: generated from input)
  --ref REF             reference genome .fa (build .fai index with samtools faidx) (required for mod bams)
  --motif MOTIF         expected modification motif (e.g. CG for 5mCpG required for mod bams)
  --max_read_density MAX_READ_DENSITY
                        filter reads with call density greater >= value, can be helpful in footprinting assays (default=None)
  --excl_ambig          do not consider reads that align entirely within segment
  --spanning_only       only consider reads that span segment
  --primary_only        ignore non-primary alignments
  --phased              currently only considers two phases (diploid)

In [ ]:
##Now we have the .tsv file and we can do segplot
methylartist segplot -s IE11_multiplex_segmeth.tsv -d m -o IE11_multiplex_segmeth.jpg


In [ ]:
##We will try to generate the plot with methylartist composite. Generates "composite" methylation plots where multiple per-element profiles are aligned to and plotted against a reference sequence.
##Ejemplo:
    
$ methylartist composite -d MCF7_data_megalodon.txt -s L1HS.bed -r Homo_sapiens_assembly38.fasta -t L1.3.fa  -p 32 --blocks L1.3.highlights.bed

The contents of L1HS.bed are formatted as follows (first 10 lines), derived from hg38 repeatmasker .out files:

chr1    34566056        34572105        L1HS    -
chr1    67078892        67084915        L1HS    -
chr1    71513699        71519742        L1HS    +
chr1    80939204        80945257        L1HS    -
chr1    84052390        84058406        L1HS    -
chr1    85748520        85754548        L1HS    +
chr1    85927068        85933100        L1HS    +
chr1    86679081        86685111        L1HS    -
chr1    104770248       104776278       L1HS    -
chr1    104843834       104849864       L1HS    -

The consensus sequence for establishing a common coordinate system is in L1.3.fa (fasta formatted with one sequence), available here: L19088.1 (it contains 6000 sequence of the human LINE-1 (L1Hs) element).

The contents of L1.3.highlights.bed used for optional highlighting of regions (ORF1 and ORF2 in this case). 


##We can try:

$ methylartist composite -b /storage/scratch01/groups/bu/teloseq/methylartist/data/IE11_multiplex/IE11supv435mC6mAfilter_aligned.bam -s /storage/scratch01/groups/bu/teloseq/methylartist/data/chr1_7_3000.bed  -r /storage/scratch01/groups/bu/teloseq/methylartist/data/hg002v1.1.fasta -p 32 -o /storage/scratch01/groups/bu/teloseq/methylartist/output/IE11_multiplex/IE11_multiplex_composite
    
(methylartist) [mespejo@cluster1-head1 methylartist]$ methylartist composite -h
2025-01-23 09:50:06,060 starting methylartist with command: /home/mespejo/miniforge3/envs/methylartist/bin/methylartist composite -h
usage: methylartist composite [-h] [-d DATA] [-b BAMS] -s SEGDATA -r REF -t TEREF [-p PROCS] [-c PALETTE] [-a ALPHA] [-w LINEWIDTH]
                              [-l LENFRAC] [-q MIN_MAPQ] [-o OUTFILE] [--meanplot_ylabel MEANPLOT_YLABEL]
                              [--meanplot_cutoff MEANPLOT_CUTOFF] [--mod MOD] [--motif MOTIF] [--blocks BLOCKS] [--start START]
                              [--end END] [--mincalls MINCALLS] [--minelts MINELTS] [--maxelts MAXELTS]
                              [--slidingwindowsize SLIDINGWINDOWSIZE] [--slidingwindowstep SLIDINGWINDOWSTEP]
                              [--smoothwindowsize SMOOTHWINDOWSIZE] [--smoothfunc SMOOTHFUNC] [--ymin YMIN] [--ymax YMAX]
                              [--max_read_density MAX_READ_DENSITY] [--excl_ambig] [--primary_only] [--phased] [--color_by_phase]
                              [--output_table] [--svg]

options:
  -h, --help            show this help message and exit
  -d DATA, --data DATA  text file with .bam filename and corresponding methylation database per line(whitespace-delimited)
  -b BAMS, --bams BAMS  one or more .bams with MM and ML tags for modification calls (see samtags spec)
  -s SEGDATA, --segdata SEGDATA
                        BED3+1: chrom, start, end, strand
  -r REF, --ref REF     ref genome fasta
  -t TEREF, --teref TEREF
                        TE ref fasta
  -p PROCS, --procs PROCS
                        multiprocessing
  -c PALETTE, --palette PALETTE
                        palette for samples (default = "tab10"), see https://seaborn.pydata.org/tutorial/color_palettes.html
  -a ALPHA, --alpha ALPHA
                        alpha (default: 0.3)
  -w LINEWIDTH, --linewidth LINEWIDTH
                        line width (default: 1)
  -l LENFRAC, --lenfrac LENFRAC
                        fraction of TE length that must align (default 0.95)
  -q MIN_MAPQ, --min_mapq MIN_MAPQ
                        minimum mapping quality (mapq), default = 10
  -o OUTFILE, --outfile OUTFILE
                        output file name (default: generated from input)
  --meanplot_ylabel MEANPLOT_YLABEL
                        set y-axis label on mean plot
  --meanplot_cutoff MEANPLOT_CUTOFF
                        override site coverage cutoff for mean plot (see output for automatic value)
  --mod MOD             modification to plot (mod codes will be listed, default: infer from sample name
  --motif MOTIF         modified motif to highlight (default = CG)
  --blocks BLOCKS       blocks to highlight (txt file with start, end, name, hex colour)
  --start START         start plotting at this base (default None)
  --end END             end plotting at this base (default None)
  --mincalls MINCALLS   minimum call count to include elt (default = 100)
  --minelts MINELTS     minimum output elements (default = 1)
  --maxelts MAXELTS     maximum output elements, if > max random.sample() (default = 200)
  --slidingwindowsize SLIDINGWINDOWSIZE
                        size of sliding window for meth frac (default 10)
  --slidingwindowstep SLIDINGWINDOWSTEP
                        step size for meth frac (default 1)
  --smoothwindowsize SMOOTHWINDOWSIZE
                        size of window for smoothing (default 8)
  --smoothfunc SMOOTHFUNC
                        smoothing function, one of: flat,hanning,hamming,bartlett,blackman (default = hanning)
  --ymin YMIN           y-axis minimum for smoothed plot
  --ymax YMAX           y-axis maximum for smoothed plot
  --max_read_density MAX_READ_DENSITY
                        filter reads with call density greater >= value, can be helpful in footprinting assays (default=None)
  --excl_ambig
  --primary_only        ignore non-primary alignments
  --phased              currently only considers two phases (diploid)
  --color_by_phase      color samples by HP value (req --phased)
  --output_table        output per-site data to table (.tsv)
  --svg
